In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
from collections import Counter
import pandas as pd

class BitFlipConfigRecovery:
    def __init__(self, valid_codewords):
        """
        valid_codewords: list of bitstrings, e.g. ['000','111']
        """
        self.valid = valid_codewords
        self.freq = Counter()

    def repair_and_update(self, raw_counts):
        recovered = Counter()
        # accumulate raw frequencies
        for bits, cnt in raw_counts.items():
            self.freq[bits] += cnt
        # repair each sample to nearest valid codeword
        for bits, cnt in raw_counts.items():
            best = min(self.valid, key=lambda v: sum(a != b for a, b in zip(bits, v)))
            recovered[best] += cnt
        return recovered

# 3-qubit bit-flip code encoding
def encode_bit_flip_code():
    qr = QuantumRegister(3, 'q')
    cr = ClassicalRegister(3, 'c')
    qc = QuantumCircuit(qr, cr)
    # prepare logical |+> on data qubit
    qc.h(qr[0])
    # encode repetition code
    qc.cx(qr[0], qr[1])
    qc.cx(qr[0], qr[2])
    # measure all
    qc.measure(qr, cr)
    return qc


def main():
    # 1) Build encode circuit
    qc = encode_bit_flip_code()

    # 2) Run on IBM QPU 'ibm_aachen'
    service = QiskitRuntimeService()
    backend = service.backend(name="ibm_aachen")
    qc_t = transpile(qc, backend=backend, optimization_level=3)
    with Session(backend=backend) as session:
        sampler = Sampler(backend)
        job = sampler.run([qc_t], shots=2048)
        raw_counts = job.result()[0].join_data().get_counts()

    # 3) Dynamic Configuration Recovery
    valid = ['000', '111']
    dcr = BitFlipConfigRecovery(valid)
    recovered = dcr.repair_and_update(raw_counts)

    # 4) Display results
    df = pd.DataFrame([{'bitstring': b,
                        'raw': raw_counts.get(b, 0),
                        'recovered': recovered.get(b, 0)}
                       for b in sorted(set(raw_counts) | set(recovered))])
    print(df.to_string(index=False))

if __name__ == '__main__':
    main()


base_primitive._get_mode_service_backend:WARNING:2025-07-01 02:27:07,803: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.


bitstring  raw  recovered
      000 1025       1054
      001    5          0
      010   15          0
      011    8          0
      100    9          0
      101  179          0
      110    7          0
      111  800        994
